In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/r7user5/Desktop/STAT')
import comonotonic as cm
import ensemble_ciber as ec
import math
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import copy
import utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import ensemble_ciber as ec
import conditional_ciber as cc
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTENC
import xlsxwriter
from sklearn.metrics import roc_auc_score
import statistics
import math

/home/r7user5/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('oil_spill.csv')
df.drop(df.columns[[0,22]], axis = 1, inplace=True)

In [3]:
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
df.columns = colnames

In [4]:
categorical = []
cont_col = [i for i in range(df.shape[1]-1)]
discrete_feature_val = None

In [5]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df[['Y']])

In [6]:
scaler = preprocessing.StandardScaler()
scale_col_name = ["X"+str(i) for i in cont_col]
df_train[scale_col_name] = scaler.fit_transform(df_train[scale_col_name])
reduced_df_train = utils.outlier_removal(df_train, cont_col)
df_test[scale_col_name] = scaler.transform(df_test[scale_col_name])

/home/r7user5/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/r7user5/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/home/r7user5/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [7]:
x_train = reduced_df_train.iloc[:,:-1].to_numpy()
y_train = reduced_df_train.iloc[:,-1].to_numpy()
smote = SMOTE(random_state=0)
x_train_synthetic, y_train_synthetic = smote.fit_resample(x_train, y_train)
x_test = df_test.iloc[:,:-1].to_numpy()
y_test = df_test.iloc[:,-1].to_numpy()

In [8]:
# return {model: [acc1...acc5]} and {model: [auc1...auc5]}
def change_train_size(X_train_full, Y_train_full, X_test, Y_test, train_size, repeat = 10):
    categorical = []
    cont_col = [i for i in range(X_train_full.shape[1])]
    discrete_feature_val = None
    acc_record = {'CIBer':[],'NB':[],'XGBoost':[],'Random Forest':[],'Decision Tree':[],'SVM':[],'Logistic Regression':[]}
    auc_record = {'CIBer':[],'NB':[],'XGBoost':[],'Random Forest':[],'Decision Tree':[],'SVM':[],'Logistic Regression':[]}
    for i in range(repeat):
        train_idx = np.random.randint(X_train_full.shape[0], 
                                      size=int((train_size/0.7)*X_train_full.shape[0]))
        X_train, Y_train = X_train_full[train_idx,:], Y_train_full[train_idx]
        # ciber
        ciber = cm.clustered_comonotonic(X_train,Y_train,discrete_feature_val,cont_col,categorical, 
                                         0.993, None, corrtype='mutual_info', discrete_method='mdlp')
        ciber.run()
        ciber_predict = ciber.predict(X_test)
        ciber_proba = ciber.predict_proba(X_test)[:,1]
        acc_record['CIBer'].append(accuracy_score(Y_test, ciber_predict))
        auc_record['CIBer'].append(roc_auc_score(Y_test, ciber_proba))
        del ciber, ciber_predict, ciber_proba
        print("CIBer at the " + str(i+1) + "-th repeat")
        # nb
        gaussian_nb = GaussianNB()
        gaussian_nb.fit(X_train, Y_train)
        nb_predict = gaussian_nb.predict(X_test)
        nb_proba = gaussian_nb.predict_proba(X_test)[:,1]
        acc_record['NB'].append(accuracy_score(Y_test, nb_predict))
        auc_record['NB'].append(roc_auc_score(Y_test, nb_proba))
        del gaussian_nb, nb_predict, nb_proba
        print("NB at the " + str(i+1) + "-th repeat")
        # xgb
        xgb_clf = xgb.XGBClassifier()
        xgb_clf.fit(X_train, Y_train)
        xgb_predict = xgb_clf.predict(X_test)
        xgb_proba = xgb_clf.predict_proba(X_test)[:,1]
        acc_record['XGBoost'].append(accuracy_score(Y_test, xgb_predict))
        auc_record['XGBoost'].append(roc_auc_score(Y_test, xgb_proba))
        del xgb_clf, xgb_predict, xgb_proba
        print("XGB at the " + str(i+1) + "-th repeat")
        # rf
        rf_clf = RandomForestClassifier()
        rf_clf.fit(X_train, Y_train)
        rf_predict = rf_clf.predict(X_test)
        rf_proba = rf_clf.predict_proba(X_test)[:,1]
        acc_record['Random Forest'].append(accuracy_score(Y_test, rf_predict))
        auc_record['Random Forest'].append(roc_auc_score(Y_test, rf_proba))
        del rf_clf, rf_predict, rf_proba
        print("RF at the " + str(i+1) + "-th repeat")
        # DT
        DT_clf = tree.DecisionTreeClassifier()
        DT_clf.fit(X_train, Y_train)
        DT_predict = DT_clf.predict(X_test)
        DT_proba = DT_clf.predict_proba(X_test)[:,1]
        acc_record['Decision Tree'].append(accuracy_score(Y_test, DT_predict))
        auc_record['Decision Tree'].append(roc_auc_score(Y_test, DT_proba))
        del DT_clf, DT_predict, DT_proba
        print("DT at the " + str(i+1) + "-th repeat")
        # svm
        svm = SVC(probability=True)
        svm.fit(X_train, Y_train)
        svm_predict = svm.predict(X_test)
        svm_proba = svm.predict_proba(X_test)[:,1]
        acc_record['SVM'].append(accuracy_score(Y_test, svm_predict))
        auc_record['SVM'].append(roc_auc_score(Y_test, svm_proba))
        del svm, svm_predict, svm_proba
        print("SVM at the " + str(i+1) + "-th repeat")
        # lr
        lr = LogisticRegression()
        lr.fit(X_train, Y_train)
        lr_predict = lr.predict(X_test)
        lr_proba = lr.predict_proba(X_test)[:,1]
        acc_record['Logistic Regression'].append(accuracy_score(Y_test, lr_predict))
        auc_record['Logistic Regression'].append(roc_auc_score(Y_test, lr_proba))
        del lr, lr_predict, lr_proba
        print("LR at the " + str(i+1) + "-th repeat")
        del train_idx, X_train, Y_train
    return acc_record, auc_record

In [9]:
def compute_CI(ll):
    mean = statistics.mean(ll)
    std = statistics.stdev(ll)
    t_value = 2.262
    lower = mean - t_value * std / math.sqrt(9)
    upper = mean + t_value * std / math.sqrt(9)
    return lower, upper

In [10]:
# from {model: [acc1...acc5]} and {model: [auc1...auc5]} to {model:{train_size: [acc1...acc5]}}
trainset_size = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
acc_book = {'CIBer':{},'NB':{},'XGBoost':{},'Random Forest':{},'Decision Tree':{},'SVM':{},'Logistic Regression':{}}
auc_book = {'CIBer':{},'NB':{},'XGBoost':{},'Random Forest':{},'Decision Tree':{},'SVM':{},'Logistic Regression':{}}
for train_size in trainset_size:
    print("Train size: "+ str(train_size))
    acc_record, auc_record = change_train_size(x_train_synthetic, y_train_synthetic, x_test, y_test,train_size)
    for model in acc_record.keys():
        acc_book[model][train_size] = acc_record[model]
        auc_book[model][train_size] = auc_record[model]

Train size: 0.1
CIBer at the 1-th repeat
NB at the 1-th repeat
XGB at the 1-th repeat
RF at the 1-th repeat
DT at the 1-th repeat
SVM at the 1-th repeat


/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR at the 1-th repeat
CIBer at the 2-th repeat
NB at the 2-th repeat
XGB at the 2-th repeat
RF at the 2-th repeat
DT at the 2-th repeat
SVM at the 2-th repeat
LR at the 2-th repeat


/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CIBer at the 3-th repeat
NB at the 3-th repeat
XGB at the 3-th repeat
RF at the 3-th repeat
DT at the 3-th repeat
SVM at the 3-th repeat


...

/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF at the 10-th repeat
DT at the 10-th repeat


/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM at the 10-th repeat
LR at the 10-th repeat


/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
# write {model:{train_size: [acc1...acc5]}} to file
workbook1 = xlsxwriter.Workbook('Oil Spill change_train_size_acc.xlsx')
workbook2 = xlsxwriter.Workbook('Oil Spill change_train_size_auc.xlsx')
worksheet1 = workbook1.add_worksheet()
worksheet2 = workbook2.add_worksheet()
title1 = ['Train size','Acc1','Acc2','Acc3','Acc4','Acc5','Acc6','Acc7','Acc8','Acc9','Acc10','T-Lower','T-Upper']
title2 = ['Train size','auc1','auc2','auc3','auc4','auc5','auc6','auc7','auc8','auc9','auc10','T-Lower','T-Upper']
row = 0
for model in acc_book.keys():
    worksheet1.write(row, 0, model)
    worksheet2.write(row, 0, model)
    row += 1
    
    for col in range(len(title1)):
        worksheet1.write(row, col, title1[col])
        worksheet2.write(row, col, title2[col])
    row += 1
    
    for train_size in trainset_size:
        worksheet1.write(row, 0, train_size)
        worksheet2.write(row, 0, train_size)
        low1, up1 = compute_CI(acc_book[model][train_size])
        low2, up2 = compute_CI(auc_book[model][train_size])
        worksheet1.write(row, 11, low1)
        worksheet1.write(row, 12, up1)
        worksheet2.write(row, 11, low2)
        worksheet2.write(row, 12, up2)
        for col in range(10):
            worksheet1.write(row, col+1, acc_book[model][train_size][col])
            worksheet2.write(row, col+1, auc_book[model][train_size][col])
        row += 1
workbook1.close()
workbook2.close()